## Cálculo da emissão e distância percorrida 

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [15]:
import pandas as pd
import random
from utils.save_csv import save_csv
from utils.load_csv import load_csv
from datetime import datetime

In [16]:
consumo_diesel = load_csv('consumo_diesel.csv')

veiculos = load_csv('veiculos.csv')

posicoes_df = load_csv('posicoes_onibus.csv')

In [17]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]
consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [18]:
def get_tipo_onibus(id_onibus):

    onibus = veiculos[veiculos['id_veiculo']==id_onibus]

    return onibus['modelo'].unique()[0]

In [19]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2

In [20]:
data_frame = []

for id_onibus in posicoes_df['id_onibus'].unique():
    df_onibus = posicoes_df[posicoes_df['id_onibus'] == id_onibus].reset_index(drop=True)
    df_onibus.sort_values(by='t', inplace=True)
    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['x']
            x_depois = df_onibus.loc[i+1, 'x']
            y_atual = row['y']
            y_depois = df_onibus.loc[i+1, 'y']
            momento_inicial = row['t']
            momento_final = df_onibus.loc[i+1, 't']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_diesel = get_fator_consumo(tipo_veiculo)
            
            #mas tem que reprojetar para SIRGAS 2000
            dist = (x_depois - x_atual)**2 + (y_depois - y_atual)**2
            
            dados = {
                'id_onibus' : row['id_onibus'],
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_diesel_km' : litro_diesel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

df_final = pd.DataFrame(data_frame)

In [21]:
df_final = df_final.merge(
    veiculos[['id_veiculo', 'eletrico']],
    left_on='id_onibus',
    right_on='id_veiculo',
    how='left').drop(columns='id_veiculo')

In [22]:
dados_trips_layer = []

for onibus in df_final['id_onibus'].unique():

    df_onibus = df_final[df_final['id_onibus'] == onibus].reset_index(drop=True)
    coordinates = []
    timestamps = []
    emissao_total = 0
    df_onibus.sort_values(by='momento_inicial', inplace=True)
    for i, row in df_onibus.iterrows():
        coordinates.append(row['ponto_inicial'])
        stamp = datetime.fromisoformat(row['momento_inicial']).timestamp()
        timestamps.append(stamp)
        emissao_total += row['emissao_co2']

    dados_trips_layer.append({
        'id_onibus': onibus,
        'emissao_co2': emissao_total,
        'eletrico': row['eletrico'],
        'coordinates' : coordinates,
        'timestamps' : timestamps
    })


df_trips = pd.DataFrame(dados_trips_layer)

In [23]:
def init_on_zero(timestamp_list)->list[float]:

    inicial = timestamp_list[0]

    new_timestamps = [ts - inicial for ts in timestamp_list]

    return [int(ts) for ts in new_timestamps]

df_trips['timestamps'] = df_trips['timestamps'].apply(init_on_zero)

In [24]:
coord_t = str(df_trips.loc[0, 'coordinates'])

In [25]:
eval(coord_t)

[[-46.56242941038247, -23.51361415867532],
 [-46.56042941038247, -23.51161415867532],
 [-46.55842941038247, -23.509614158675323],
 [-46.556429410382464, -23.507614158675324],
 [-46.55442941038246, -23.505614158675325],
 [-46.55242941038246, -23.50361415867533],
 [-46.550429410382456, -23.501614158675327],
 [-46.54842941038245, -23.499614158675328],
 [-46.54642941038245, -23.49761415867533],
 [-46.54442941038245, -23.49561415867533],
 [-46.54242941038245, -23.49361415867533],
 [-46.54042941038245, -23.491614158675333],
 [-46.53842941038244, -23.489614158675334],
 [-46.53642941038244, -23.48761415867533],
 [-46.53442941038244, -23.485614158675336],
 [-46.532429410382434, -23.483614158675337],
 [-46.53042941038243, -23.481614158675335],
 [-46.52842941038243, -23.47961415867534],
 [-46.52642941038243, -23.47761415867534],
 [-46.524429410382425, -23.47561415867534],
 [-46.52242941038242, -23.473614158675343],
 [-46.52042941038242, -23.471614158675344],
 [-46.51842941038242, -23.469614158675

In [26]:
df_trips.loc[0, 'timestamps']

[0,
 60,
 120,
 180,
 240,
 300,
 360,
 420,
 480,
 540,
 600,
 660,
 720,
 780,
 840,
 900,
 960,
 1020,
 1080,
 1140,
 1200,
 1260,
 1320,
 1380,
 1440,
 1500,
 1560,
 1620,
 1680,
 1740,
 1800,
 1860,
 1920,
 1980,
 2040,
 2100,
 2160,
 2220,
 2280]

In [27]:
df_trips = df_trips[['coordinates', 'timestamps']]

In [28]:
save_csv(df_final, 'df_final.csv')

save_csv(df_trips, 'df_trips.csv')

Base salva em data\df_final.csv
Base salva em data\df_trips.csv
